# ABS Quarterly Financial Accounts 5232

## Python set-up

In [1]:
# system imports
import textwrap

# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol

# local imports
from abs_helper import get_abs_data, get_gdp
from mgplot import line_plot_finalise, multi_start

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False
FILE_TYPE = "png"

## Get data from ABS

In [2]:
abs_dict, meta, source, _ = get_abs_data("5232.0")
plot_times = 0, -17
plot_tags = "", "-recent"

In [3]:
# list tables
textwrap.wrap(", ".join(abs_dict.keys()), width=80)

['5232001, 5232002, 5232003, 5232004, 5232005, 5232006, 5232007, 5232008, 5232009,',
 '5232010, 5232011, 5232012, 5232013, 5232014, 5232015, 5232016, 5232017, 5232018,',
 '5232019, 5232020, 5232021, 5232022, 5232023, 5232024, 5232025, 5232026, 5232027,',
 '5232028, 5232029, 5232030, 5232031, 5232032, 5232033, 5232034, 5232035, 5232036,',
 '5232037, 5232038, 5232039, 5232040, 5232041, 5232042, 5232043, 5232044, 5232045,',
 '5232046, 5232047, 5232048, 5232049, 5232050, 5232051, 5232052, 5232053, 5232054']

## Plot

### Household balance sheet

In [4]:
def plot_hbs() -> None:
    """Plot the household balance sheet data."""

    table = "5232035"
    data = abs_dict[table]
    series_type = "Original"
    plots = meta[meta["Table"] == table][metacol.did]
    gdp, gdp_units = get_gdp("CP", "O")
    print(f"Original GDP units: {gdp_units}") 
    gdp /= 1_000  # convert from millions to billions

    for plot in plots:
        search = {series_type: metacol.stype, plot: metacol.did, table: metacol.table}
        _table, ident, orig_units = ra.find_abs_id(meta, search)

        # manage long wordy chart titles
        max_length = 60
        title = f"HBS: {plot[:-2]}"
        footnote = ""
        replaceable = {
            "- Produced - Fixed assets -": ["- PFA -", " PFA = Produced fixed assets."],
            "- Non-produced assets -": ["- NPA -", " NPA = Non-produced assets."],
            "- Non-financial -": ["- NF -", " NF = Non-financial."],
            "- Fixed assets -": ["- FA -", " FA = Fixed assets."],
            "Financial assets -": ["FA -", " FA = Financial assets."],
        }
        for foundling, [replacement, fnote] in replaceable.items():
            if foundling in title and len(title) >= max_length:
                title = title.replace(foundling, replacement)
                footnote = f"{footnote}{fnote}"
        title = textwrap.fill(title, width=max_length)

        orig_series = data[ident]
        series, units = ra.recalibrate(orig_series, orig_units)
        print(f"Series units: {orig_units}")

        # plot
        multi_start(
            series,
            function=line_plot_finalise,
            annotate=True,
            starts=plot_times,
            title=title,
            ylabel=f"{units} current prices",
            rfooter=source,
            lfooter=f"Australia. {series_type.capitalize()} series."
            + f" HBS = Household balance sheet.{footnote}",
            width=2,
            show=SHOW,
            file_type=FILE_TYPE,
        )

        line_plot_finalise(
            orig_series / gdp.rolling(4, min_periods=4).sum() * 100,
            title=f"{title} (% GDP 4Q sum)",
            ylabel=f"Per cent GDP current prices",
            width=2,
            annotate=True,
            rfooter=f"{source} 5206.0",
            lfooter=f"Australia. {series_type.capitalize()} series. "
            + f" HBS = Household balance sheet.{footnote}. "
            + "GDP = Original series, current prices, 4Q rolling sum. ",
            show=SHOW,
            file_type=FILE_TYPE,
        )


plot_hbs()

Original GDP units: $ Millions
Series units: $ Billions


Series units: $ Billions


Series units: $ Billions


Series units: $ Billions


Series units: $ Billions


...

Series units: $ Billions


Series units: $ Billions


Series units: $ Billions


Series units: $ Billions


Series units: $ Billions


In [5]:
def plot_lad() -> None:
    """Plot the household balance sheet data for land and dwellings."""

    # Land and dwelling assets - side by side
    table = "5232035"
    data = abs_dict[table]
    series_type = "Original"
    land = "Non-financial - Non-produced assets - Land ;"
    dwellings = "Non-financial - Produced - Fixed assets - Dwellings ;"
    combined = {}
    for name, series in zip(("Land", "Dwellings"), (land, dwellings)):
        search = {series_type: metacol.stype, series: metacol.did, table: metacol.table}
        _table, ident, units = ra.find_abs_id(meta, search)
        combined[name] = data[ident]
    cdf = pd.DataFrame(combined)
    cdf, units = ra.recalibrate(cdf, units)
    title = "Household Balance Sheet for Land and Dwellings"
    line_plot_finalise(
        cdf,
        title=title,
        ylabel=f"{units} current prices",
        rfooter=source,
        lfooter=f"Australia. {series_type.capitalize()} series. ",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
        annotate=True,
    )


plot_lad()

In [6]:
def real_net_worth() -> None:
    """Adjust Net Worth for inflation and population growth."""

    # net worth
    table = "5232035"
    series_id = "A83722648X"
    nw, nw_units = abs_dict[table][series_id], "$ Billions"

    # get NA data
    cat = "5206.0"
    kags = "5206001_Key_Aggregates"
    ipd = "5206005_Expenditure_Implicit_Price_Deflators"
    wanted = {
        "A2302460K": (cat, kags, "GDP per capita: Chain volume measures (Orig)"),
        "A2302459A": (
            cat,
            kags,
            "Gross domestic product: Chain volume measures (Orig)",
        ),
        "A2303940R": (cat, ipd, "Households Final Consumption Expenditure (SA)"),
    }
    df = pd.DataFrame()
    for series_id, (category_id, seo, title) in wanted.items():
        d, _m = ra.read_abs_series(category_id, series_id, single_excel_only=seo)
        df[title] = d[series_id]

    # calculate population then average net wealth per capita
    population = (
        df["Gross domestic product: Chain volume measures (Orig)"]
        / df["GDP per capita: Chain volume measures (Orig)"]
    ) * 1_000_000
    ave_wealth, units = ra.recalibrate(nw / population, nw_units)
    line_plot_finalise(
        ave_wealth,
        title="Average Net Wealth per Capita",
        ylabel=f"{units} current prices",
        rfooter=f"{source} {category_id}",
        lfooter="Australia. " + "Population from National Accounts. ",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
        annotate=True,
    )

    # adjust net wealth for inflation
    base = df["Households Final Consumption Expenditure (SA)"].iloc[-1]
    deflate = df["Households Final Consumption Expenditure (SA)"] / base
    real = ave_wealth / deflate
    line_plot_finalise(
        # real["2019Q4":],
        real,
        title="Real Average Net Wealth per Capita",
        ylabel=f"{units} {df.index[-1]} prices",
        rfooter=f"{source} {category_id}",
        lfooter="Australia. "
        + "Inflation adjusted using the HFCE deflator. "
        + "Population from National Accounts. ",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
        annotate=True,
    )


real_net_worth()

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-09-04 08:22:30

Python implementation: CPython
Python version       : 3.13.6
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

readabs: 0.1.4
pandas : 2.3.1
mgplot : 0.2.12

Watermark: 2.5.0



In [8]:
print("Finished")

Finished
